In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://www.dropbox.com/s/ddkmtqz01jc024u/glove.6B.100d.txt

--2023-03-22 05:50:22--  https://www.dropbox.com/s/ddkmtqz01jc024u/glove.6B.100d.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/ddkmtqz01jc024u/glove.6B.100d.txt [following]
--2023-03-22 05:50:23--  https://www.dropbox.com/s/raw/ddkmtqz01jc024u/glove.6B.100d.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc85606899ade1dd9b0e5f512172.dl.dropboxusercontent.com/cd/0/inline/B4vZa-6A3X0gxrDVWCumRv7cEmKlZHKdW4GUGuEB_yg2v1m-tlqcB1_RrM2nOkHhHE-PQplxPj4KrfhYAAiZpVdLtRfgrmnqpQxdqL90RrxTuyTm5D9OZ0PS_0R7KqMu7seuUoWLS1KyRTV5D1gXQZojmYU1FIM7_O_fyDd0q7vGFQ/file# [following]
--2023-03-22 05:50:23--  https://uc85606899ade1dd9b0e5f512172.dl.dropboxusercontent.com/cd/0/inline/B4vZa-6A3X0gxrDVWCumRv7cEmKlZHKdW4GUGuEB_yg2v1m-tlqcB1_RrM2nOkHhH

<font color='blue'>**Load the data**</font>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
# import seaborn as sns
import pandas as pd
import re
from tqdm.auto import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import pandas as pd
import numpy as np
import nltk
import string
import time
import re
import gzip
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import datetime
import pickle
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import json
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.keras.backend.clear_session()
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,concatenate,Activation,Dropout,Input
from tensorflow.keras.models import Model
import keras
import keras.utils
from tensorflow.keras import utils as np_utils
from tensorflow.keras import models
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, LSTM, Dropout, BatchNormalization, Dense, concatenate
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from keras.utils import plot_model
from IPython.display import Image
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.bleu_score import sentence_bleu
from prettytable import PrettyTable

In [ ]:
# with open('ita.txt', 'r', encoding="utf8") as f:
#     eng=[]
#     ita=[]
#     for i in f.readlines():
#         eng.append(i.split("\t")[0])
#         ita.append(i.split("\t")[1])
# data = pd.DataFrame(data=list(zip(eng, ita)), columns=['english','italian'])
# print(data.shape)
# data.head()

In [ ]:
#http://jmcauley.ucsd.edu/data/amazon/qa/

def parse(path):
     g = gzip.open(path, 'rb')
     for l in g:
         yield eval(l)

def getDF(path):
     i = 0
     df = {}
     for d in parse(path):
         df[i] = d
         i += 1
     return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
%%time
DF1 = getDF('/content/sample_data/qa_Automotive.json.gz')
DF2 = getDF('/content/sample_data/qa_Baby.json.gz')
DF3 = getDF('/content/sample_data/qa_Beauty.json.gz')
DF4 = getDF('/content/sample_data/qa_Cell_Phones_and_Accessories.json.gz')
DF5 = getDF('/content/sample_data/qa_Clothing_Shoes_and_Jewelry.json.gz')
DF6 = getDF('/content/sample_data/qa_Electronics.json.gz')
DF7 = getDF('/content/sample_data/qa_Grocery_and_Gourmet_Food.json.gz')
DF8 = getDF('/content/sample_data/qa_Health_and_Personal_Care.json.gz')
DF9 = getDF('/content/sample_data/qa_Home_and_Kitchen.json.gz')
DF10 = getDF('/content/sample_data/qa_Musical_Instruments.json.gz')
DF11 = getDF('/content/sample_data/qa_Office_Products.json.gz')
DF12 = getDF('/content/sample_data/qa_Patio_Lawn_and_Garden.json.gz')
DF13 = getDF('/content/sample_data/qa_Pet_Supplies.json.gz')
DF14 = getDF('/content/sample_data/qa_Sports_and_Outdoors.json.gz')
DF15 = getDF('/content/sample_data/qa_Tools_and_Home_Improvement.json.gz')
DF16 = getDF('/content/sample_data/qa_Video_Games.json.gz')
DF17 = getDF('/content/sample_data/qa_Toys_and_Games.json.gz')

DF = pd.concat([DF1,DF2,DF3,DF4,DF5,DF6,DF7,DF8,DF9,DF10,DF11,DF12,DF13,DF14,DF15,DF16,DF17],
               sort=False,axis=0)
DF

In [ ]:
DF.rename(columns = {'question':'questionText'}, inplace = True)
DF.rename(columns = {'answer':'answerText'}, inplace = True)

In [ ]:
data = DF[['questionText','answerText']]

In [ ]:
data.shape

(1343851, 2)

In [ ]:
data = pd.concat([data]*2)
print(data.shape)

(2687702, 2)


## <font color='blue'>**Preprocess data**</font>

In [ ]:
def decontractions(phrase):
     #Reference: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)
    return phrase

def preprocess(text):
    # https://gist.github.com/anandborad/d410a49a493b56dace4f814ab5325bbd
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    return text

def preprocess_ita(text):
    text = text.lower()
    text = decontractions(text)
    text = re.sub('[$)\?"’.°!;\'€%:,(/]', '', text)
    text = re.sub('\u200b', ' ', text)
    text = re.sub('\xa0', ' ', text)
    text = re.sub('-', ' ', text)
    return text


data['questionText'] = data['questionText'].apply(preprocess)
data['answerText'] = data['answerText'].apply(preprocess_ita)
data.head()

,questionText,answerText
0,what is the most useful length to get,at least 20 feetheres whysay you have to pull ...
1,are these cables made of copper or aluminum,coleman is website does indeed say copper equi...
2,i bought the red extra heavy duty is that too ...,for jumper cables you can never have too much ...
3,hi being 20ft 4gauge how heavy is this,not nearly heavy enough i keep them under my b...
4,do these cables come with a bag,no


In [ ]:
que_lengths = data['questionText'].str.split().apply(len)
ans_lengths = data['answerText'].str.split().apply(len)

If you observe the values, 99.9% of the data points are having length < 22 for Italian sentences and length < 25 for English sentences , so select the sentences that have words < 22 and < 25 respectively.

Inorder to do the teacher forcing while training of seq-seq models, lets create two new columns, one with token at begining of the sentence and other column with token at the end of the sequence

In [ ]:
data['que_len'] = data['questionText'].str.split().apply(len)
data = data[data['que_len'] <= 11]

data['ans_len'] = data['answerText'].str.split().apply(len)
data = data[data['ans_len'] <= 3]

data['ans_inp'] = '<start> ' + data['answerText'].astype(str)
data['ans_out'] = data['answerText'].astype(str) + ' <end>'

data = data.drop(['answerText','que_len','ans_len'], axis=1)
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
data.head()

Inorder to do the teacher forcing while training of seq-seq models, lets create two new columns, one with <start> token at begining of the sentence and other column with <end> token at the end of the sequence

## Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(data, test_size=0.2,random_state=9)

In [ ]:
print(train.shape,validation.shape)

(159316, 3) (39830, 3)


In [ ]:
train.iloc[0]

questionText    can you use a remote to change channels
ans_inp                                      <start> no
ans_out                                        no <end>
Name: 134164, dtype: object

In [ ]:
import joblib
joblib.dump((train,validation),'/content/sample_data/train_test.pkl')

['/content/sample_data/train_test.pkl']

In [ ]:
train,validation = joblib.load('/content/sample_data/train_test.pkl')

In [ ]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['ans_inp']= str(train.iloc[0]['ans_inp'])+' <end>'
train.iloc[0]['ans_out']= str(train.iloc[0]['ans_out'])+' <end>'

(159316, 3) (39830, 3)


In [ ]:
validation.head(2)

,questionText,ans_inp,ans_out
81295,does this case include a snap on belt clip,<start> yes it did,yes it did <end>
123274,does this computer work well with gaming,<start> yes,yes <end>


## Creating Tokenizer on the train data and learning vocabulary
Note that we are fitting the tokenizer only on train data and check the filters for english, we need to remove symbols except < and >

In [ ]:
tknizer_ita = Tokenizer()
tknizer_ita.fit_on_texts(train['questionText'].values)
tknizer_eng = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')
tknizer_eng.fit_on_texts(train['ans_inp'].values)

In [ ]:
joblib.dump((tknizer_ita,tknizer_eng),'/content/sample_data/tokenizer.pkl')

['/content/sample_data/tokenizer.pkl']

In [ ]:
tknizer_ita,tknizer_eng = joblib.load('/content/sample_data/tokenizer.pkl')

In [ ]:
vocab_size_ans=len(tknizer_eng.word_index.keys())
print(vocab_size_ans)
vocab_size_que=len(tknizer_ita.word_index.keys())
print(vocab_size_que)

10308
29031


In [ ]:
tknizer_eng.word_index['<start>'], tknizer_eng.word_index['<end>']

(1, 8163)

In [ ]:
eng_dict = {}
for key, value in tknizer_eng.word_index.items():
  eng_dict[value] = key

In [ ]:
tknizer_eng.word_index['<start>'], tknizer_eng.word_index['<end>']

(1, 8163)

In [ ]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix = np.zeros((vocab_size_ans+1, 100))
for word, i in tknizer_eng.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## <font color='blue'>**Implement custom encoder decoder**</font>

<font color='blue'>**Encoder**</font>

In [ ]:
class Encoder(tf.keras.Model):

    def __init__(self,inp_vocab_size,embedding_size,enc_units,input_length):

        super(Encoder,self).__init__()
        self.inp_vocab_size = inp_vocab_size
        self.enc_units = enc_units
        #Initialize Embedding layer
        self.embedding = Embedding(input_dim = inp_vocab_size, output_dim = embedding_size, input_length = input_length, name="embedding_layer_encoder")
        
        #Intialize Encoder LSTM layer
        self.lstm = LSTM(enc_units,return_state=True,return_sequences=True,name="Encoder_LSTM")


    def call(self,input_sequence,states):

        input_embed = self.embedding(input_sequence)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(
            input_embed,initial_state = [states[0], states[1]])
                
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c


    def initialize_states(self,batch_size):
      self.state_h = tf.zeros((batch_size, self.enc_units))
      self.state_c = tf.zeros((batch_size, self.enc_units))

      return self.state_h ,self.state_c
      


In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size,embedding_size,dec_units,input_length):
        super(Decoder, self).__init__()
        self.out_vocab_size = out_vocab_size
     
        #Initialize Embedding layer
        self.embedding = Embedding(input_dim = out_vocab_size, output_dim = embedding_size, input_length = input_length, name="embedding_layer_decoder")

        #Intialize Decoder LSTM layer
        self.lstm = LSTM(dec_units, return_sequences=True, return_state=True, name="Decoder_LSTM")
        


    def call(self,target_sentances,initial_states):
   
        target_embedd = self.embedding(target_sentances)
        self.decoder_output,self.decoder_hidden_state,self.decoder_cell_state = self.lstm(
            target_embedd, initial_state = [initial_states[0], initial_states[1]])
        
        return self.decoder_output, self.decoder_hidden_state, self.decoder_cell_state

      


In [ ]:
class Encoder_decoder(tf.keras.Model):
    
    def __init__(self,encoder_inputs_length,decoder_inputs_length,encoder_vocab_size,decoder_vocab_size,encoder_embed_size,decoder_embed_size,output_vocab_size,):
        super().__init__()
        
        #Create encoder object
        self.encoder = Encoder(inp_vocab_size = encoder_vocab_size, embedding_size = encoder_embed_size, input_length = encoder_inputs_length, enc_units=256)
        #Create decoder object
        self.decoder = Decoder(out_vocab_size = decoder_vocab_size,embedding_size= decoder_embed_size,input_length = decoder_inputs_length,dec_units=256)
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        self.dense = Dense(output_vocab_size, activation='softmax')
        
    
    def call(self,data):
    
        encoder_input,decoder_input = data[0], data[1]
        #Intializing encoder initial states
        initial_state = self.encoder.initialize_states(1024)
        encoder_output, encoder_h, encoder_c = self.encoder(encoder_input,initial_state)
        decoder_output,_,_ = self.decoder(decoder_input, [encoder_h, encoder_c])
        output = self.dense(decoder_output)
        return output
        
        

In [ ]:
#calculate FP, FN, TP, TN wrt text data

## Creating data pipeline

In [ ]:
class Dataset:
    def __init__(self, data, tknizer_ita, tknizer_eng, max_len):
        self.encoder_inps = data['questionText'].values
        self.decoder_inps = data['ans_inp'].values
        self.decoder_outs = data['ans_out'].values
        self.tknizer_eng = tknizer_eng
        self.tknizer_ita = tknizer_ita
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_ita.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_eng.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_eng.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [ ]:
train_dataset = Dataset(train, tknizer_ita, tknizer_eng, 25)
test_dataset  = Dataset(validation, tknizer_ita, tknizer_eng, 25)

train_dataloader = Dataloder(train_dataset, batch_size=1024)
test_dataloader = Dataloder(test_dataset, batch_size=1024)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(1024, 25) (1024, 25) (1024, 25)


In [ ]:
test_dataloader[0][0][0]

array([[   4,    1,   39, ...,    0,    0,    0],
       [   4,    1,  213, ...,    0,    0,    0],
       [  10,   24,  398, ...,    0,    0,    0],
       ...,
       [  10,   26,   30, ...,    0,    0,    0],
       [   9,    4, 4319, ...,    0,    0,    0],
       [   3,    2, 4171, ...,    0,    0,    0]], dtype=int32)

In [ ]:
# this is the same model we have given in the other reference notebook
class MyModel(Model):
    def __init__(self, encoder_inputs_length,decoder_inputs_length, output_vocab_size):
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        self.encoder = Encoder(vocab_size=vocab_size_que+1, embedding_dim=50, input_length=encoder_inputs_length, enc_units=256)
        self.decoder = Decoder(vocab_size=vocab_size_ans+1, embedding_dim=100, input_length=decoder_inputs_length, dec_units=256)
        self.dense   = Dense(output_vocab_size, activation='softmax')  
    def call(self, data):
        input,output = data[0], data[1]
        encoder_output, encoder_h, encoder_c = self.encoder(input)
        decoder_output                       = self.decoder(output, encoder_h, encoder_c)
        output                               = self.dense(decoder_output)
        return output

In [ ]:
model = Encoder_decoder(encoder_inputs_length=22,decoder_inputs_length=25,encoder_vocab_size = vocab_size_que + 1,
                         decoder_vocab_size = vocab_size_ans + 1,encoder_embed_size = 100,decoder_embed_size = 100,output_vocab_size = vocab_size_ans)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#tensorboard = TensorBoard(log_dir='/content/sample_data/logs', histogram_freq=1,write_graph=True)
# tensorboard callback
log_dir = '/content/sample_data/logs'
tensorboard_cb = TensorBoard(log_dir=log_dir)
model.fit(train_dataloader, steps_per_epoch=train_steps, epochs=10, validation_data=train_dataloader, validation_steps=valid_steps,callbacks = tensorboard_cb)
model.summary()

In [ ]:
model = Encoder_decoder(encoder_inputs_length=22,decoder_inputs_length=25,encoder_vocab_size = vocab_size_que + 1,
                         decoder_vocab_size = vocab_size_ans + 1,encoder_embed_size = 100,decoder_embed_size = 100,output_vocab_size = vocab_size_ans)
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer,loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
train_steps=train.shape[0]//1024
valid_steps=validation.shape[0]//1024
#tensorboard = TensorBoard(log_dir='/content/sample_data/logs', histogram_freq=1,write_graph=True)
# tensorboard callback
log_dir = '/content/sample_data/logs'
tensorboard_cb = TensorBoard(log_dir=log_dir)
model.fit(train_dataloader, steps_per_epoch=train_steps, epochs=10, validation_data=test_dataloader, validation_steps=valid_steps,callbacks = tensorboard_cb)
model.summary()

In [ ]:
# make lookup table
eng_index_word={}
eng_word_index={}
for key,value in tknizer_eng.word_index.items():
  eng_index_word[value]=key
  eng_word_index[key]=value

In [ ]:
eng_dict = {}
for key, value in tknizer_eng.word_index.items():
  eng_dict[value] = key

In [ ]:
def predict(input_sentence):


  sent_token = tknizer_ita.texts_to_sequences([input_sentence])
  sent_token = pad_sequences(sent_token,maxlen=20,padding='post')

  
  initial_states = model.layers[0].initialize_states(1)
  encoder_out, h_state , c_state = model.layers[0](sent_token,initial_states)

  states = [h_state , c_state]

  sent = ''

  target_word = np.array([[1]])
  for t in range(25):
    predicted_out,state_h,state_c=model.layers[1](target_word,states)
    predicted_out = model.layers[2](predicted_out)

    states = [state_h,state_c]
    target_word = np.reshape(np.argmax(predicted_out), (1, 1))

    #sent = ''+ eng_dict[int(target_word)]
    sent += eng_dict[int(target_word)]
    if int(target_word) == tknizer_eng.word_index['<end>']:
      break

  return sent


In [ ]:
from keras.preprocessing.text import text_to_word_sequence
text_to_word_sequence('will this fit a 2008 a frontier 6 bed')

['will', 'this', 'fit', 'a', '2008', 'a', 'frontier', '6', 'bed']

In [ ]:
train

**Post training analysis:**

In [155]:
orig_sent = []
question = []
for i in range(2000):
  train_ans = train['ans_out'].iloc[i]
  orig_sent.append(train_ans.replace(" <end>",""))
  question.append(train['questionText'].iloc[i])

In [ ]:
pred_sent = []
for i in range(2000):
  prediction = predict(train['questionText'].iloc[i]).split()
  for j in prediction:
    prediction += j
    pred_sent.extend(j.replace('<end>',''))

In [146]:
import collections
def f1_score(true,pred):
  true_tokens = text_to_word_sequence(true)
  pred_tokens = text_to_word_sequence(pred)
  common = collections.Counter(true_tokens) & collections.Counter(pred_tokens)
  num_same = sum(common.values())
  if len(true_tokens) ==0 or len(pred_tokens) == 0:
    return int(true_tokens == pred_tokens)

  if num_same == 0:
    return 0

  precision = 1.0 * num_same/ len(pred_tokens)
  recall = 1.0 * num_same/len(true_tokens)
  f1_score = (2*precision*recall) /(precision + recall)
  return f1_score

In [156]:
df = pd.DataFrame(list(zip(question, orig_sent,pred_sent)), columns = ['question','original', 'predicted'])

In [161]:
f1_scores = []
for index, row in df.iterrows():
      f1 = f1_score(row['original'], row['predicted'])
      f1_scores.append(f1)

In [163]:
df['f1_scores'] = f1_scores

In [165]:
rating = []
for row in df['f1_scores']:
  if row >= 0 and row <= 0.3 :    rating.append('bad')
  elif row > 0.3 and row <=0.7 :   rating.append('good')
  elif row > 0.7:  rating.append('best')

  else:           
    rating.append('Not_Rated')

In [167]:
df['rating'] = rating

In [172]:
df.head(50)

,question,original,predicted,f1_scores,rating
616,does it come with a base as in the image online,yes it does,yes it does,1.0,best
1872,will it fit a eos rebel t2i with a 1855mm lens,yes,yes,1.0,best
1002,is this a 2014 calendar,yes,yes,1.0,best
1369,does it come with a microphone,yes,yes,1.0,best
1248,is this lamp electronic or magnetic model,electrc,electrc,1.0,best
1731,does this work with android phones also,yes and blackberry,yes,0.5,good
1414,is this pretempered chocolate,yes,yes,1.0,best
663,what hose do you use for thr tail of the polaris,tail hose,tail hose,1.0,best
1340,will these tapes fit into a car stereo,yes,yes,1.0,best
222,wattage draw,62 amps,62 amps,1.0,best


We could see that we have taken a sample of 2000 datapoints from the big dataset, where we have calculated the f1_scores and gave the ratings based on it.


If the score value lied between 0 and 0.3 we rated as bad, if 0.3 to 0.7 good and if it was more than 0.7 we gave it as best. Also we could infer from the above values is that the model performed well where we had yes/no choice of answers, when it came to predicting any number of value other than y/n it could not perform that much good for e.g., 'when is the expiration date', it was supposed to give date value but we instead got yes as output.